In [2]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\laimi\.nuget\packages\plotly.net.interactive\4.2.1\lib\netstandard2.1\Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

let trade_csv = "d:\\studies\\03_export_date_ticker_screenerid_gap_outcomes.csv"

let csv = studies.GapStudy.parseTradeOutcomes trade_csv

In [3]:
let outputSummary includeChart (summary:studies.Types.TradeSummary) = 

    printfn "%s: %i trades, %.2f%% win pct, %.2f%% avg gain, %.2f%% avg loss, %.2f avg gain/loss, %.2f EV" 
        summary.StrategyName summary.Total (summary.WinPct * 100m) (summary.AvgWin) (summary.AvgLoss) summary.AvgGainLoss summary.EV

    match includeChart with
    | true ->
        let chart = 
            Chart.Histogram(
                X = summary.Gains,
                NBinsX = 40,
                Name = "Gain Distribution"
            )

        chart.Display() |> ignore
    | false -> ()

In [4]:
let filterNamePairs = [
    ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
    ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
    ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
    ("Gap ups", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp)
    ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 28)
    ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.HasGapUp && o.Screenerid = 29)
]

let tradesGroupedByStrategy = csv |> Seq.groupBy (fun t -> t.Strategy) // all strategies, no filters

filterNamePairs
|> Seq.iter (
    fun (filterName,filter) ->

        printfn $"Trade: {filterName}"
        tradesGroupedByStrategy
        |> Seq.iter (
            fun (strategy, trades) ->
                let filteredTrades = trades |> Seq.filter filter
                let summary = studies.Types.TradeSummary.create filteredTrades
                outputSummary false summary
        )

        printfn ""
        printfn ""
)

Trade: All
Buy and Hold 5 bars: 22848 trades, 50.28% win pct, 7.40% avg gain, -6.87% avg loss, 1.08 avg gain/loss, 0.30 EV
Buy and Hold 10 bars: 22848 trades, 49.16% win pct, 9.15% avg gain, -9.07% avg loss, 1.01 avg gain/loss, -0.12 EV
Buy and Hold 30 bars: 22848 trades, 45.54% win pct, 14.08% avg gain, -14.67% avg loss, 0.96 avg gain/loss, -1.58 EV
Buy and Hold 60 bars: 22848 trades, 44.82% win pct, 18.24% avg gain, -18.96% avg loss, 0.96 avg gain/loss, -2.29 EV
Buy and Hold 90 bars: 22848 trades, 44.18% win pct, 20.46% avg gain, -21.37% avg loss, 0.96 avg gain/loss, -2.89 EV
Buy and Hold: 22848 trades, 43.08% win pct, 29.77% avg gain, -30.46% avg loss, 0.98 avg gain/loss, -4.52 EV


Trade: New Highs
Buy and Hold 5 bars: 8692 trades, 47.95% win pct, 4.57% avg gain, -4.50% avg loss, 1.02 avg gain/loss, -0.15 EV
Buy and Hold 10 bars: 8692 trades, 48.48% win pct, 5.86% avg gain, -6.27% avg loss, 0.94 avg gain/loss, -0.38 EV
Buy and Hold 30 bars: 8692 trades, 45.38% win pct, 9.53% avg ga

In [5]:
let data =
    filterNamePairs
    |> Seq.map (fun (filterName, filter) ->
        tradesGroupedByStrategy
        |> Seq.map (fun (strategy, trades) ->
            let filteredTrades = trades |> Seq.filter filter
            let summary = studies.Types.TradeSummary.create filteredTrades
            (strategy, filterName, summary)
        )
    )
    |> Seq.concat

let sortFunctions = [
    ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
    ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
    ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
]

sortFunctions
|> Seq.iter (fun (sortLabel, sortFunction) ->
    printfn $"Top 5 by: {sortLabel}"
    data
    |> Seq.sortByDescending (fun (strategy, filterName, summary) -> sortFunction summary)
    |> Seq.take 5
    |> Seq.iter (fun (strategy, filterName, summary) ->
        printfn $"Strategy: {strategy}, Filter: {filterName}: {sortFunction summary}"
    )
    printfn ""
    printfn ""
)

Top 5 by: EV
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.822461023902737286555059309
Strategy: Buy and Hold, Filter: New Highs: 0.736784720611110804648903637
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5825484022408140426570786786
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5698105391586863628178409871
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.4783539464164832646621860451


Top 5 by: Win %
Strategy: Buy and Hold 5 bars, Filter: Top Gainers: 0.5171658660638598474145238768
Strategy: Buy and Hold 5 bars, Filter: Gap ups - top gainers: 0.5154989384288747346072186837
Strategy: Buy and Hold 10 bars, Filter: Gap ups - top gainers: 0.5131634819532908704883227176
Strategy: Buy and Hold 10 bars, Filter: Gap ups: 0.5081433224755700325732899023
Strategy: Buy and Hold 5 bars, Filter: Gap ups: 0.507057546145494028230184582


Top 5 by: Avg Gain
Strategy: Buy and Hold, Filter: Gap ups - new highs: 1.2635946275090736548676078349
Strategy: Buy and Hold 5 

In [2]:
#r "./bin/Debug/net7.0/studies.dll"
#r "./bin/Debug/net7.0/core.fs.dll"
#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

Installed Packages FSharp.Data, 6.3.0 Plotly.NET.Interactive, 4.2.1

In [9]:
let studiesDirectory = "d:\\studies"
let signalPath = "d:\\studies\\02_export_date_ticker_screenerid_gap.csv"
let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let runStrategies strategies = async {
    let! signalsWithPrices = studies.Trading.prepareSignalsForTradeSimulations signalPath priceFunc
    
    return! studies.Trading.runTrades signalsWithPrices strategies
}

In [11]:
let strategies = 
        [
            studies.TradingStrategies.buyAndHoldStrategy (Some 5)
            studies.TradingStrategies.buyAndHoldStrategy (Some 10)
            studies.TradingStrategies.buyAndHoldStrategy (Some 30)
            studies.TradingStrategies.buyAndHoldStrategy (Some 60)
            studies.TradingStrategies.buyAndHoldStrategy (Some 90)
            studies.TradingStrategies.buyAndHoldStrategy None
        ]

let outcomes = 
    strategies
    |> runStrategies
    |> Async.RunSynchronously

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

Records: 23520, dates: 423, tickers: 3164, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Ensured that data has prices
Records: 22848, dates: 412, tickers: 3153, screenerIds: 2
Minimum date: 3/22/2022 12:00:00 AM
Maximum date: 11/13/2023 12:00:00 AM

Executing trades...
Finished running trades, generated:
22848 trade outcomes
